In [323]:
import iris

import numpy as np
import scipy.stats

In [324]:
%load_ext autoreload
%autoreload 2
import math_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [325]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube("testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc")
cm_future = iris.load_cube("testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc")

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [326]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [327]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [328]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [329]:
from isimip import ISIMIP

temp_debiaser = ISIMIP(variable = "tas")
temp_debiaser.variable = "prsnratio"
"""temp_debiaser.trend_preservation = "mixed"
temp_debiaser.lower_bound = 0
temp_debiaser.upper_bound = 300
temp_debiaser.lower_threshold = 50
temp_debiaser.upper_treshold = 290
temp_debiaser.powerlaw_exponent_step4 = 2"""

'temp_debiaser.trend_preservation = "mixed"\ntemp_debiaser.lower_bound = 0\ntemp_debiaser.upper_bound = 300\ntemp_debiaser.lower_threshold = 50\ntemp_debiaser.upper_treshold = 290\ntemp_debiaser.powerlaw_exponent_step4 = 2'

In [330]:
obs = obs[obs > 0] # Two obs-outliers. Need to check for that.

In [331]:
obs, cm_hist, cm_future = temp_debiaser.step2(obs, cm_hist, cm_future)
obs, cm_hist, cm_future = temp_debiaser.step4(obs, cm_hist, cm_future)
obs_future = temp_debiaser.step5(obs, cm_hist, cm_future)
test = temp_debiaser.step6(obs, obs_future, cm_hist, cm_future)

In [332]:
test

array([278.32870353, 276.5465267 , 281.61202083, 281.74360822,
       289.46205774, 295.23919669, 296.30356676, 297.28599871,
       292.11233843, 287.54466362, 284.53383197, 278.32604012,
       277.78250285, 278.188538  , 280.8291732 , 283.88331537,
       287.22796138, 296.51977458, 298.61395774, 297.89249953,
       294.58405619, 290.19658908, 282.49233709, 280.72829004,
       270.48202409, 279.92997589, 283.87502269, 282.23430773,
       286.17339284, 293.17472598, 298.99248908, 299.96780868,
       292.36816853, 286.32572823, 282.23769918, 279.1485278 ,
       278.35988651, 279.66832773, 283.86674426, 281.07328933,
       288.50759064, 295.12116709, 293.69206555, 297.69128473,
       294.00543455, 285.25070175, 283.17011532, 280.7309818 ,
       280.41883153, 278.75653282, 282.79096906, 284.49410637,
       290.29832369, 298.06018676, 297.27805626, 295.5228383 ,
       296.01012193, 288.19676364, 284.02573989, 280.49292511,
       278.00621905, 281.47297239, 283.14825252, 288.09